In [1]:
import hickle as hkl
import numpy as np

from lib.read_data import u_desvio as u_true
from lib.read_data import y_desvio as y_true

ny = y_true.shape[1]
nu = u_true.shape[1]
theta = hkl.load("../export/ARX.hkl")

A = theta[:ny, :].T
B = theta[ny:, :].T


def model(y: np.ndarray, u_ext: np.ndarray):
    if y.shape != (ny,):
        raise ValueError(f"y inválido: shape {y.shape}, esperado {(ny,)}")

    if u_ext.shape != (2 * nu,):
        raise ValueError(f"u_ext inválido: shape {u_ext.shape}, esperado {(2 * nu,)}")

    return A @ y + B @ u_ext  # y(k+1)

print(A.shape, B.shape)


(3, 3) (3, 8)


In [2]:
from scipy.optimize import minimize

from lib.read_data import u_max_desvio as u_max
from lib.read_data import u_min_desvio as u_min

Np = 10  # horizonte de predição
Nc = 3  # horizonte de controle
Q = np.full(ny, 1.0)  # peso no erro
R = np.full(nu, 0.1)  # peso na variação da entrada
ki = np.full(ny, 0.0001)  # peso do integrador


def J(u_seq, y0, sp, u0, ni):
    """
    u_seq: sequência de entradas (Nc,)
    y0: saída atual
    sp: setpoint
    u0: última entrada aplicada
    """
    u_seq = u_seq.reshape((Nc, nu))
    y = y0
    cost = 0.0

    u_old = u0
    for k in range(Np):
        # Usa a última entrada se k >= Nc
        if k < Nc:
            u = u_seq[k]
        else:
            u = u_seq[-1]

        u_ext = np.concatenate([u_old, u])

        y_next = model(y, u_ext)
        e = y_next - sp + ni  # shape (ny,)

        cost += e.T @ np.diag(Q) @ e

        u_old = u

    u_old = u0
    for k in range(Nc):
        u = u_seq[k]

        du = u - u_old
        cost += du.T @ np.diag(R) @ du

        u_old = u

    return cost


def MPC(y0, sp, u_old, ni):
    """
    y0: saída atual
    sp: setpoint
    u_old: última entrada aplicada
    """
    u0 = np.tile(u_old, Nc).flatten()  # chute inicial: repete u_old Nc vezes

    # Restrições
    bounds_step = list(zip(u_min, u_max))
    bounds = bounds_step * Nc

    # Otimização
    res = minimize(J, u0, args=(y0, sp, u_old, ni), method="SLSQP", bounds=bounds)

    # Retorna a primeira entrada da sequência otimizada
    return np.array(res.x).reshape((Nc, nu))[0]


# Simulando malha fechada


In [3]:
ncycles = 150
cycles = np.arange(0, ncycles)

# Prealocando histórico de saída e entrada
y = np.zeros((ncycles, ny))
y[0] = y_true[0]
u = np.zeros((ncycles, nu))
u[0] = u_true[0]

y_sp = np.zeros((ncycles, ny))  # setpoint y
y_sp[:] = np.array([2.60473585, -8.24649468, 8.04911466])

u_sp = np.zeros((ncycles, nu))  # setpoint u
u_sp[:] = np.array([-70, -45, 5, 25])

ni = 0
for k in range(1, ncycles):
    ni += (y[k - 1] - y_sp[k]) * ki
    # print(ni)

    u[k] = MPC(y[k - 1], y_sp[k], u[k - 1], ni)
    # u[k] = u_sp

    # Atualiza o estado
    y[k] = model(y[k - 1], np.concatenate([u[k - 1], u[k]]))


In [4]:
from lib.read_data import u_ref, y_ref

# u_ref = np.zeros_like(u_ref)
# y_ref = np.zeros_like(y_ref)

y = y + np.tile(y_ref, (y.shape[0], 1))
y_sp = y_sp + y_ref

u = u + np.tile(u_ref, (u.shape[0], 1))
u_sp = u_sp + u_ref

print(y_sp.shape)


(150, 3)


In [5]:
# Plotando gráficos
from lib.plot import plot_control

plot_control(cycles, u, y, y_sp, u_sp, model_name="MPC")
